In [1]:
import pandas as pd

### Leer los dataframes

- geo = geolocation
- prop = property-detail

In [2]:
prop = pd.read_csv("..\datos\property-detail_last price.csv", sep=";")


### Eliminar viviendas que no son de Valencia

In [3]:
geo["indexes"] = geo["location_text"].str.find("Valencia")
#geo["location_text"], geo["indexes"]
geo["indexes"] = geo["indexes"]>=0
geo["indexes"]

geo = geo.drop(geo[geo["indexes"]==False].index)
geo

,id,location_text,type,parent,location_id,child,indexes
1131,24777,"Barx, Valencia",Municipality,0-EU-ES-46-04,0-EU-ES-46-04-010-046,0,True
1133,24779,"Benifairo de la Valldigna, Valencia",Municipality,0-EU-ES-46-04,0-EU-ES-46-04-010-059,0,True
1134,24780,"Xeraco, Valencia",Municipality,0-EU-ES-46-04,0-EU-ES-46-04-009-143,0,True
1136,24782,"Xeresa, Valencia",Municipality,0-EU-ES-46-04,0-EU-ES-46-04-009-146,0,True
1139,24785,"Simat de la Valldigna, Valencia",Municipality,0-EU-ES-46-04,0-EU-ES-46-04-010-231,0,True
...,...,...,...,...,...,...,...
10360,34607,"En Corts, Valencia",Neighbourhood,0-EU-ES-46-02-002-250-10,0-EU-ES-46-02-002-250-10-002,0,True
10361,34608,"Malilla, Valencia",Neighbourhood,0-EU-ES-46-02-002-250-10,0-EU-ES-46-02-002-250-10-003,0,True
11243,36122,"Nou Campanar, Valencia",Neighbourhood,0-EU-ES-46-02-002-250-04,0-EU-ES-46-02-002-250-04-006,0,True
11244,36123,"Nou Benicalap, Valencia",Neighbourhood,0-EU-ES-46-02-002-250-16,0-EU-ES-46-02-002-250-16-003,0,True


### Comparar Variables

- Propertycode + ad_id $\rightarrow$ Misma variable
- Ver cuántas veces se repite -1 en price_last (12 veces)
- Ver cuántos anuncios no han sido actualizados (ad_statistics) $\rightarrow$ 1022
    - En realidad no quiere decir que no hayan sido actualizados, sino que no hay info.
- Comprobar si "batch_name" es categorical o no $\rightarrow$ diría que sí.
- Ver la estructura del diccionario de "utag_data":
    - Contiene info redundante de que ya ha sido dividida en las variables
- Comprobar si "url" y "url_source" contienen lo mimso $\rightarrow$ no.
    - "url" $\rightarrow$ dirección url actual de la vivienda.
    - "url_source" $\rightarrow$ dirección desactualizada de la vivienda (no sirve de nada)
    - !!! comprobar si alguna de las variables de url source tiene status_code 200
- Comprobar si "ad_address_locationLevel" y "ad_geo_locationLevel" contienen lo mismo $\rightarrow$ sí

In [4]:
(prop['ad_id'] != prop['propertycode']).sum()

(prop["price_last"] == -1).sum()

(prop["ad_statistics"].str.find("Anuncio actualizado")<0).sum()

prop["batch_name"].value_counts()

prop["utag_data"][0]

(prop['url'] == prop['url_source']).sum()

(prop['ad_address_locationLevel'] != prop['ad_geo_locationLevel']).sum()

0

In [5]:
prop["ad_geo_type"].isnull().sum()

0

In [6]:
prop["ad_geo_type"]

0        OFFICIAL_ZONE
1        OFFICIAL_ZONE
2        OFFICIAL_ZONE
3        OFFICIAL_ZONE
4        OFFICIAL_ZONE
             ...      
35479    OFFICIAL_ZONE
35480    OFFICIAL_ZONE
35481    OFFICIAL_ZONE
35482    OFFICIAL_ZONE
35483    OFFICIAL_ZONE
Name: ad_geo_type, Length: 35484, dtype: object

In [7]:
prop["ad_geo_type"].value_counts()

OFFICIAL_ZONE    35484
Name: ad_geo_type, dtype: int64

In [66]:
lista1 = []
lista2 = []
for i in prop[prop["ad_owner_type"].isnull()]:
    lista1.append(prop["propertycode"])
for i in prop[prop["advertizer_type"].isnull()]:
    lista2.append(prop["propertycode"])
    
print(lista1 == lista2)


[0          194584
1          297019
2          298885
3          299168
4          300605
           ...   
35479    95533035
35480    95533054
35481    95533371
35482    95533401
35483    95533443
Name: propertycode, Length: 35484, dtype: int64, 0          194584
1          297019
2          298885
3          299168
4          300605
           ...   
35479    95533035
35480    95533054
35481    95533371
35482    95533401
35483    95533443
Name: propertycode, Length: 35484, dtype: int64, 0          194584
1          297019
2          298885
3          299168
4          300605
           ...   
35479    95533035
35480    95533054
35481    95533371
35482    95533401
35483    95533443
Name: propertycode, Length: 35484, dtype: int64, 0          194584
1          297019
2          298885
3          299168
4          300605
           ...   
35479    95533035
35480    95533054
35481    95533371
35482    95533401
35483    95533443
Name: propertycode, Length: 35484, dtype: int64, 0          

In [8]:
print(prop[["date_last", "date_update"]])
prop[prop["date_last"] == prop["date_update"]]
#prop["date_update"].isnull().sum()

                        date_last                 date_update
0      2022-02-21 18:39:34.209749  2022-01-16 01:23:16.373611
1      2022-02-23 13:49:56.335920  2021-12-17 23:17:18.450535
2      2022-02-23 12:38:29.031511  2022-01-12 04:29:31.440967
3      2022-02-23 12:30:22.814937  2022-01-12 02:47:55.547618
4      2022-02-23 12:05:59.625347  2021-12-30 11:39:08.418644
...                           ...                         ...
35479  2022-02-12 14:20:01.455868  2022-02-09 10:42:53.207473
35480  2022-02-23 12:53:49.283372  2022-01-12 12:32:51.747990
35481  2022-02-23 13:40:25.045597  2021-12-17 21:35:49.932825
35482  2022-02-23 13:25:50.958539  2021-12-17 15:45:52.343189
35483  2022-02-23 14:08:19.748437  2021-12-18 03:47:35.819507

[35484 rows x 2 columns]


,propertycode,active,ad_address_locationId,ad_address_locationLevel,ad_address_municipalityId,ad_address_provinceId,ad_builtType,ad_characteristics_bathNumber,ad_characteristics_constructedArea,ad_characteristics_hasGarden,...,url,utag_data,batch_name,ad_statistics,advertizer_id_name,ad_condition_isNewDevelopment,ad_energyCertification,url_source,price_last,date_last


## ELIMINACIÓN DE VARIABLES E INDIVIDUOS

In [17]:
propsinvar = prop.drop(["ad_builtType", "ad_address_municipalityId", "propertycode", "active", "ad_address_locationId", "ad_address_locationLevel",
           "ad_address_provinceId", "ad_energyCertification_sufix", 
          "ad_geo_locationId", "ad_geo_type", "ad_hasRecommended", "ad_id", "ad_isRecommended", 
          "ad_numberRecommended", "ad_origin", "ad_originTypeRecommended", "ad_owner_commercialId",
          "ad_owner_commercialName", "ad_owner_contactPreference", "ad_price",
          "ad_recommendationId", "ad_typeRecommended", "advertizer_contact_name", 
          "advertizer_loc", "advertizer_logo", "advertizer_office_name", "advertizer_type", 
          "advertizer_url", "deactivated", "deactivated_reason", "features", "image_urls", 
          "phone", "photo_main", "reason", "success", "title", "utag_data", "batch_name", 
          "ad_statistics", "advertizer_id_name", "ad_condition_isNewDevelopment", "ad_energyCertification",
          "url_source", "ad_media_has3Dtour", "ad_media_hasHomeStaging", "ad_media_hasFloorPlan",
            "ad_media_photoNumber", "ad_media_videoNumber", "ad_geo_locationLevel", "address_hidden"], axis =1)

In [18]:
len(propsinvar.columns)

22

In [19]:
prop1 = propsinvar[propsinvar["price_last"] != -1]

In [20]:
len(prop1.columns)
prop1.columns

Index(['ad_characteristics_bathNumber', 'ad_characteristics_constructedArea',
       'ad_characteristics_hasGarden', 'ad_characteristics_hasLift',
       'ad_characteristics_hasParking', 'ad_characteristics_hasSwimmingPool',
       'ad_characteristics_hasTerrace', 'ad_characteristics_roomNumber',
       'ad_condition_isGoodCondition', 'ad_condition_isNeedsRenovating',
       'ad_operation', 'ad_owner_type', 'ad_typology', 'address_location',
       'date_insert', 'date_update', 'latitude', 'longitude', 'price', 'url',
       'price_last', 'date_last'],
      dtype='object')

#### Eliminar individuos:
- Comprobar si tienen demasiados valores faltantes

In [55]:
#propsinna = porp1.dropna(thresh=22)
#propsinna

,ad_address_municipalityId,ad_builtType,ad_characteristics_bathNumber,ad_characteristics_constructedArea,ad_characteristics_hasGarden,ad_characteristics_hasLift,ad_characteristics_hasParking,ad_characteristics_hasSwimmingPool,ad_characteristics_hasTerrace,ad_characteristics_roomNumber,...,address_hidden,address_location,date_insert,date_update,latitude,longitude,price,url,price_last,date_last
1,0-EU-ES-46-15-011-613,3.0,1.0,70.0,1.0,1.0,1.0,1.0,1.0,2.0,...,0,"[""Calle Caravelas, 8"", ""Playa de Farnals"", ""L'...",2021-12-17 23:17:18.450499,2021-12-17 23:17:18.450535,39.563871,-0.286502,155000,https://www.idealista.com/inmueble/297019/,164000,2022-02-23 13:49:56.335920
2,0-EU-ES-46-04-002-131,3.0,2.0,180.0,0.0,1.0,0.0,0.0,0.0,4.0,...,0,"[""Calle Jeresa, 17"", ""Barrio Corea"", ""Distrito...",2022-01-12 04:29:31.440967,2022-01-12 04:29:31.440967,38.972413,-0.182877,90000,https://www.idealista.com/inmueble/298885/,90000,2022-02-23 12:38:29.031511
3,0-EU-ES-46-04-002-131,3.0,2.0,162.0,1.0,1.0,1.0,1.0,1.0,3.0,...,0,"[""Avenida Grau, 60"", ""Distrito Playa de Gandia...",2022-01-12 02:47:55.547618,2022-01-12 02:47:55.547618,38.997958,-0.161492,245000,https://www.idealista.com/inmueble/299168/,245000,2022-02-23 12:30:22.814937
4,0-EU-ES-46-01-003-082,3.0,2.0,170.0,0.0,1.0,1.0,0.0,1.0,3.0,...,0,"[""Calle Palancia, 8"", ""Canet d'En Berenguer"", ...",2021-12-30 11:39:08.418644,2021-12-30 11:39:08.418644,39.679193,-0.218145,275000,https://www.idealista.com/inmueble/300605/,275000,2022-02-23 12:05:59.625347
5,0-EU-ES-46-04-002-131,3.0,2.0,115.0,0.0,1.0,1.0,0.0,1.0,3.0,...,0,"[""Calle Convent, 1"", ""Distrito Benipeixcar - E...",2022-01-12 05:52:35.560854,2022-01-12 05:52:35.560854,38.960679,-0.190555,97000,https://www.idealista.com/inmueble/301211/,97000,2022-02-23 12:39:17.081998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35477,0-EU-ES-46-06-002-229,1.0,3.0,210.0,1.0,NaN,1.0,0.0,0.0,4.0,...,0,"[""Calle El Ramal , 212"", ""Siete Aguas"", ""Hoya ...",2021-12-18 06:13:47.512097,2021-12-18 06:13:47.512148,39.473788,-0.916398,140000,https://www.idealista.com/inmueble/95533004/,140000,2021-12-18 05:09:25.540013
35480,0-EU-ES-46-07-008-093,3.0,1.0,300.0,1.0,NaN,NaN,1.0,NaN,5.0,...,1,"[""Catadau"", ""Ribera Alta, València""]",2022-01-12 12:32:51.747990,2022-01-12 12:32:51.747990,39.275335,-0.580692,149900,https://www.idealista.com/inmueble/95533054/,149900,2022-02-23 12:53:49.283372
35481,0-EU-ES-46-15-012-207,3.0,1.0,70.0,0.0,0.0,NaN,0.0,NaN,3.0,...,0,"[""Calle Castelló, 1 b"", ""Rafelbunyol"", ""L'Hort...",2021-12-17 21:35:49.932787,2021-12-17 21:35:49.932825,39.587564,-0.331475,63000,https://www.idealista.com/inmueble/95533371/,63000,2022-02-23 13:40:25.045597
35482,0-EU-ES-46-17-003-194,3.0,2.0,127.0,0.0,1.0,NaN,0.0,1.0,3.0,...,0,"[""Calle Sant Joan Ribera, 34 -40"", ""Picassent""...",2021-12-17 15:45:52.343140,2021-12-17 15:45:52.343189,39.358389,-0.466377,113000,https://www.idealista.com/inmueble/95533401/,113000,2022-02-23 13:25:50.958539


In [43]:
propsinna["ad_characteristics_hasLift"].isna().sum()

12125

In [38]:
propsinna[propsinna["ad_characteristics_hasLift"].isna()]

,ad_address_municipalityId,ad_builtType,ad_characteristics_bathNumber,ad_characteristics_constructedArea,ad_characteristics_hasGarden,ad_characteristics_hasLift,ad_characteristics_hasParking,ad_characteristics_hasSwimmingPool,ad_characteristics_hasTerrace,ad_characteristics_roomNumber,...,address_hidden,address_location,date_insert,date_update,latitude,longitude,price,url,price_last,date_last
7,0-EU-ES-46-08-001-145,3.0,1.0,110.0,1.0,NaN,1.0,1.0,1.0,3.0,...,0,"[""Camino la Cambra, 198"", ""Xátiva"", ""La Coster...",2021-12-17 09:19:22.395352,2021-12-17 09:19:22.395392,38.988309,-0.518566,148000,https://www.idealista.com/inmueble/303510/,148000,2022-02-23 13:17:54.151011
10,0-EU-ES-46-13-001-144,3.0,1.0,480.0,0.0,NaN,0.0,0.0,0.0,4.0,...,0,"[""Calle Escuela, 15"", ""Jarafuel"", ""El Valle de...",2021-12-18 04:54:20.214326,2021-12-18 04:54:20.214359,39.135879,-1.073927,180000,https://www.idealista.com/inmueble/304082/,180000,2022-02-23 14:11:13.234046
11,0-EU-ES-46-03-002-105,3.0,3.0,160.0,1.0,NaN,1.0,0.0,0.0,4.0,...,0,"[""Calle Barranc Fondo, 33"", ""Distrito El Faro""...",2022-01-12 17:17:23.782148,2022-01-12 17:17:23.782148,39.182766,-0.229809,300000,https://www.idealista.com/inmueble/305409/,300000,2022-02-23 14:33:06.669250
14,0-EU-ES-46-06-003-111,3.0,2.0,180.0,0.0,NaN,1.0,1.0,0.0,3.0,...,0,"[""Calle Rio Genil, 1514"", ""Urb. urbanizacion c...",2021-12-18 05:20:49.783214,2021-12-18 05:20:49.783269,39.438024,-0.575897,240000,https://www.idealista.com/inmueble/306264/,240000,2022-02-23 14:16:23.499756
17,0-EU-ES-46-03-002-105,3.0,1.0,70.0,0.0,NaN,0.0,0.0,1.0,3.0,...,0,"[""Plaza Libertat, 9"", ""Distrito Pueblo de Cull...",2022-01-12 18:22:51.057290,2022-01-12 18:22:51.057290,39.164083,-0.252350,250000,https://www.idealista.com/inmueble/306760/,250000,2022-02-23 12:10:48.920844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35474,0-EU-ES-46-06-002-229,1.0,3.0,285.0,1.0,NaN,1.0,0.0,0.0,4.0,...,0,"[""Calle El Ramal , 212"", ""Siete Aguas"", ""Hoya ...",2021-12-18 06:13:46.729483,2021-12-18 06:13:46.729518,39.473788,-0.916398,206400,https://www.idealista.com/inmueble/95532941/,206400,2021-12-18 05:09:25.819480
35475,0-EU-ES-46-06-002-229,1.0,3.0,285.0,1.0,NaN,1.0,0.0,0.0,4.0,...,0,"[""Calle El Ramal , 212"", ""Siete Aguas"", ""Hoya ...",2021-12-18 05:09:31.226124,2021-12-18 05:09:31.226158,39.473788,-0.916398,206400,https://www.idealista.com/inmueble/95532975/,206400,2021-12-18 05:09:26.327130
35476,0-EU-ES-46-06-002-229,1.0,3.0,210.0,1.0,NaN,1.0,0.0,0.0,4.0,...,0,"[""Calle El Ramal , 212"", ""Siete Aguas"", ""Hoya ...",2021-12-18 06:13:46.021143,2021-12-18 06:13:46.021190,39.473788,-0.916398,138000,https://www.idealista.com/inmueble/95532981/,138000,2021-12-18 05:09:26.076995
35477,0-EU-ES-46-06-002-229,1.0,3.0,210.0,1.0,NaN,1.0,0.0,0.0,4.0,...,0,"[""Calle El Ramal , 212"", ""Siete Aguas"", ""Hoya ...",2021-12-18 06:13:47.512097,2021-12-18 06:13:47.512148,39.473788,-0.916398,140000,https://www.idealista.com/inmueble/95533004/,140000,2021-12-18 05:09:25.540013


### Descarga del nuevo dataframe en .csv

In [41]:
prop1.to_csv("property_sinvar.csv")

In [43]:
prop1.to_excel("property_sinvar.xlsx")




## Sacar garajes y locales:

-> ad_typology
1. → Vivienda:  Piso, Duplex, Atico
2. → Casa o Chalet Independiente - Adosado - Pareado
3. → Casa Rural - Casa de Pueblo- Casa Terrera - Caseron - Finca Rustica - Masia
4. → Garaje
5. → Oficinas
6. → Local - Nave
7. → Habitacion
8. → Terreno
9. → None
10. → Obra nueva
11. → None
12. → Trastero
13. → Edificio


In [21]:
prop1[prop1["ad_typology"] == 13]

,ad_address_municipalityId,ad_builtType,ad_characteristics_bathNumber,ad_characteristics_constructedArea,ad_characteristics_hasGarden,ad_characteristics_hasLift,ad_characteristics_hasParking,ad_characteristics_hasSwimmingPool,ad_characteristics_hasTerrace,ad_characteristics_roomNumber,...,address_hidden,address_location,date_insert,date_update,latitude,longitude,price,url,price_last,date_last


## densidad y colegios

In [15]:
dencol = pd.read_excel("valencia/Densidad_y_colegios.xlsx")
#dist = pd.read_excel("valencia/distritos.xlsx")

In [16]:
dencol

,Distrito,Densidad,Numero centros educativos
0,Ciutat Vella,162.90,14
1,Eixample,247.30,13
2,Extramurs,247.10,15
3,Campanar,73.80,12
4,La Saïdia,243.20,3
...,...,...,...
279,l'Olleria,258.19,0
280,la Pobla de Farnals,2245.24,0
281,la Pobla de Vallbona,750.92,0
282,Nàquera/Náquera,178.19,0


In [29]:
cont = 0
for i in range(len(prop1)):
    if prop1["address_location"].iloc[i].split(",")[-2][2:] == "Valencia":
        cont +=1
print(cont)
        

7915


In [33]:
a = prop1["address_location"].iloc[4]
a = a.split()
a = a[-1]
a = a.replace("]", "")
a = a.replace('"', "")
a

prop1["address_location"][1000]


'["Urb. MONASTERIOS", "Distrito Los Monasterios - El Picayo - Urbanizaciones", "Sagunto/Sagunt", "Camp de Morvedre, València"]'

In [34]:
import matplotlib.pyplot as plt
import geopandas as gdp

In [36]:
coordenadas = pd.DataFrame()

In [ ]:
coordenadas[""]

In [112]:
len(porp1)

35472

In [119]:
7915/35742

0.22144815623076491

In [21]:
df = prop1

In [14]:
df.columns
df["address_location"]

1        ["Calle Caravelas, 8", "Playa de Farnals", "L'...
2        ["Calle Jeresa, 17", "Barrio Corea", "Distrito...
3        ["Avenida Grau, 60", "Distrito Playa de Gandia...
4        ["Calle Palancia, 8", "Canet d'En Berenguer", ...
5        ["Calle Convent, 1", "Distrito Benipeixcar - E...
                               ...                        
35479    ["Barrio Alquerieta", "Distrito Casc Urbà", "A...
35480                 ["Catadau", "Ribera Alta, València"]
35481    ["Calle Castelló, 1 b", "Rafelbunyol", "L'Hort...
35482    ["Calle Sant Joan Ribera, 34 -40", "Picassent"...
35483    ["Calle Fermín Vilar, 7", "Distrito El Mercado...
Name: address_location, Length: 35472, dtype: object

In [22]:
df= df[df["address_location"].str.contains("Valencia")]

In [23]:
df["ad_typology"]

21       1
29       2
42       1
48       1
59       2
        ..
35453    1
35459    1
35469    1
35471    1
35472    1
Name: ad_typology, Length: 8270, dtype: int64

In [24]:
df.to_csv("../datos/valencia.csv")